In [8]:
import psycopg2
import pandas as pd

connection = psycopg2.connect(
    host="localhost",
    database="statistc",
    user="postgres",
    password="1234"
)
try:
    cursor = connection.cursor()
    postgreSQL_select_Query ='''WITH qtd_venda AS (
	SELECT
		p.nome AS produto,
		p.preco AS preco,
		m.nome AS marca,
		c.nome AS categoria,
		(SELECT COUNT(*)
			 FROM venda v
			 WHERE m.id_marca = p.id_marca
			 AND c.id_categoria = p.id_categoria
			 AND v.id_produto = p.id_produto
			 AND v.ano = 2023
			 AND v.quantidade > 200
			 AND p.preco > 10
			GROUP BY GROUPING SETS((m.id_marca, p.id_marca, c.id_categoria, v.id_venda))
		) AS quantidade_venda
	FROM produto p, marca m, categoria c
    )
    SELECT
        produto,
        preco,
        marca,
        categoria,
        CASE
            WHEN quantidade_venda IS NULL THEN 0
            ELSE quantidade_venda
        END quantidade_venda
    FROM qtd_venda'''

    cursor.execute(postgreSQL_select_Query)
    print("Conteúdo da tabela: CATEGORIA")
    records = cursor.fetchall()
    df = pd.DataFrame(records)
    print(df)
except (Exception, psycopg2.Error) as error:
    print("Erro ao conectar com o PostgreSQL: {}".format(error))
finally:
    if connection:
        cursor.close()
        connection.close()
        print('A conexão com o PostgreSQL foi fechada!')

    


Conteúdo da tabela: CATEGORIA
                 0                        1     2
0         perdigao                alimentos   600
1             luxo  materiais de construcao  7000
2            seara                alimentos   600
3  custo beneficio                  mobilia   700
4             luxo                  mobilia   250
5  custo beneficio  materiais de construcao  9000
A conexão com o PostgreSQL foi fechada!
